In [1]:
import shutil
import os
import numpy as np
import keras
from keras.applications import VGG16
from keras import models
from keras import layers
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import Adam
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import pandas as pd
import matplotlib as mplt
import re
from keras.callbacks import TensorBoard
import datetime

In [2]:
directory = r'/home/josheen/projects/def-vinay/josheen/DL/Figure13model/runs_excel_files'
dfs = []
for runs in os.listdir(directory):
    R = pd.read_excel(os.path.join(directory,runs))
    rnum = os.path.splitext(runs)[0]
    R.head()
    R['Run'] = int(rnum[1])
    R.head()
    dfs.append(R)
data = pd.concat(dfs)

In [3]:
FileName = "/home/josheen/projects/def-vinay/josheen/DL/Figure13model/mapped_data/AllRunsmapped.csv"
directory = "/home/josheen/projects/def-vinay/josheen/DL/"
df = pd.read_csv(FileName)
for index, row in df.iterrows():
    
    address = row['Address']
    new = address.replace("\\", "/")
    add_index = address.find('Figure13model')
    new_filename = os.path.join(directory, new[add_index:])
    df.at[index, 'Address'] = new_filename
    
df.to_csv('/home/josheen/projects/def-vinay/josheen/DL/saved_df.csv', index = False, header=True)

[6 5 6 ... 6 3 7]


In [52]:
df = pd.read_csv('/home/josheen/projects/def-vinay/josheen/DL/saved_df.csv')
left=115
top=35
right=243
bottom=163

train_files = []
y_train = []
dataset = []
for index, row in df.iterrows():
    if row['Element'] == 'Zinc':
        y_train.append(row['Recovery']) 
        img = load_img(row['Address'])  # this is a PIL image

        img1 = img.crop((left,top,right,bottom))

        #img2 =img1.thumbnail((128, 128))
        x = img_to_array(img1)
        #x = x.reshape((3, 128, 128))
        #x = (x - 128.0) / 128.0
        x = (x - 255.0) / 255.0
        dataset.append(x)
    else:
        pass
y_train1 = y_train

In [47]:
test = pd.read_csv('/home/josheen/projects/def-vinay/josheen/DL/saved_df.csv')
dataset = test
y_train1 = test['Recovery']

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [53]:
from sklearn.model_selection import train_test_split
num_classes = 8
categorical_train = []
for val in y_train1:
    if val <= 5:
        categorical_train.append(0)
    elif val > 5 and val <= 10:
        categorical_train.append(1)
    elif val > 10 and val<= 15:
        categorical_train.append(2)
    elif val > 15 and val<= 20:
        categorical_train.append(3)
    elif val > 20 and val<= 25:
        categorical_train.append(4)
    elif val > 25 and val <= 30:
        categorical_train.append(5)
    elif val > 30 and val <= 35:
        categorical_train.append(6)
    elif val >35:
        categorical_train.append(7)
    y_train = categorical_train

test_to_train=0.2
val_to_test=0.5

X_train, X_test, y_train, y_test = train_test_split(dataset, y_train, test_size=test_to_train, random_state=33)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=val_to_test, random_state=33)

print("Train set size: {0}, Val set size: {1}, Test set size: {2}".format(len(X_train), len(X_val), len(X_test)))

Train set size: 1237, Val set size: 155, Test set size: 155


# Pretrained CNN - VGG16 Architecture

notes: we should look into making generators, accuracy does not seem to go higher than MAE = 1.5.

In [56]:
conv_base = VGG16(weights = 'imagenet', include_top=False, input_shape=(128,128,3))
conv_base.trainable = False 
CNN = models.Sequential()
CNN.add(conv_base)
CNN.add(layers.Flatten())
CNN.add(layers.Dense(256, activation='relu'))
CNN.add(layers.Dropout(0.2))
CNN.add(layers.Dense(32, activation='relu'))
CNN.add(layers.Dropout(0.3))
CNN.add(layers.Dense(8, activation = 'softmax'))
CNN.summary()
opt = Adam(lr=0.0002, beta_1=0.5)
CNN.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                8224      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 8)                

In [54]:
CNN.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
my_callbacks = EarlyStopping(monitor="mean_absolute_error", patience=6, mode="auto")
X_val = np.array(X_val)
X_train = np.array(X_train)
y_train = np.array(y_train)
y_val = np.array(y_val)
CNN.fit(X_train, y_train, batch_size=32,epochs=75,validation_data=(X_val, y_val))
model = CNN.save('/home/josheen/projects/def-vinay/josheen/DL/')


Epoch 1/75
39/39 [==============================] - 38s 977ms/step - loss: 28.9160 - mae: 5.2469 - val_loss: 19.4727 - val_mae: 3.9258
Epoch 2/75
39/39 [==============================] - 38s 965ms/step - loss: 28.9160 - mae: 5.2469 - val_loss: 19.5938 - val_mae: 3.9414
Epoch 3/75
39/39 [==============================] - 37s 961ms/step - loss: 28.9160 - mae: 5.2469 - val_loss: 19.5938 - val_mae: 3.9414
Epoch 4/75
39/39 [==============================] - 37s 958ms/step - loss: 28.9160 - mae: 5.2469 - val_loss: 19.5146 - val_mae: 3.9297
Epoch 5/75
39/39 [==============================] - 37s 956ms/step - loss: 28.9160 - mae: 5.2469 - val_loss: 19.4727 - val_mae: 3.9258
Epoch 6/75
39/39 [==============================] - 37s 958ms/step - loss: 28.9160 - mae: 5.2469 - val_loss: 19.4727 - val_mae: 3.9258
Epoch 7/75
39/39 [==============================] - 37s 960ms/step - loss: 28.9160 - mae: 5.2469 - val_loss: 19.4727 - val_mae: 3.9258
Epoch 8/75
39/39 [==============================] - 37s

KeyboardInterrupt: 

# GAN

In [33]:
test_to_train=0.2
val_to_test=0.5
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset['Address'], y_train1, test_size=test_to_train, random_state=33)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=val_to_test, random_state=33)

train_gen = pd.DataFrame(X_train, columns=['Address'])
train_gen['Recovery'] = y_train

test_gen = pd.DataFrame(X_test, columns=['Address'])
test_gen['Recovery'] = y_test

val_gen = pd.DataFrame(X_val, columns=['Address'])
val_gen['Recovery'] = y_val

In [34]:
#consider splitting from 0-3 3-6 6-9 9-12 12-15 15-18 18-21 7 classes
num_classes = 8
for gen in [train_gen, test_gen, val_gen]:
    categorical_train = []
    for index, row in gen.iterrows():
        val = row['Recovery']
        if val <= 5:
            categorical_train.append(0)
        elif val > 5 and val <= 10:
            categorical_train.append(1)
        elif val > 10 and val<= 15:
            categorical_train.append(2)
        elif val > 15 and val<= 20:
            categorical_train.append(3)
        elif val > 20 and val<= 25:
            categorical_train.append(4)
        elif val > 25 and val <= 30:
            categorical_train.append(5)
        elif val > 30 and val <= 35:
            categorical_train.append(6)
        elif val >35:
            categorical_train.append(7)
    gen['Recovery'] = categorical_train
    

In [6]:
print(train_gen['Recovery'][20])

3


In [6]:
train_datagen = ImageDataGenerator(
    rescale = 1./255, rotation_range = 5, width_shift_range = 0.05, height_shift_range = 0.05, shear_range=0.05,
    zoom_range=0.08, horizontal_flip = True, fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_dataframe(train_gen, directory = None, x_col = 'Address', y_col = 'Recovery', target_size = (128,128),class_mode = 'raw')
test_generator = test_datagen.flow_from_dataframe(test_gen, directory = None, x_col = 'Address', y_col = 'Recovery', target_size = (128,128), class_mode = 'raw')
val_generator = val_datagen.flow_from_dataframe(val_gen, directory = None, x_col = 'Address', y_col = 'Recovery', target_size = (128,128), class_mode = 'raw')

Found 2055 validated image filenames.
Found 257 validated image filenames.
Found 257 validated image filenames.


In [27]:
#discriminator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras import Model 

initialWeights = keras.initializers.RandomNormal(mean=0.0, stddev=0.02, seed=None)
num_of_classes = 8
in_shape = (128,128,3)
in_image = Input(shape = in_shape)
fe = Conv2D(64, (3,3), strides=(2, 2), padding='same', kernel_initializer=initialWeights)(in_image)
fe = LeakyReLU(alpha=0.2)(fe)
fe = Conv2D(64, (4,4), strides=(2, 2), padding='same', kernel_initializer=initialWeights)(fe)
fe = BatchNormalization()(fe)
fe = Conv2D(64, (4,4), strides=(2, 2), padding='same', kernel_initializer=initialWeights)(fe)
fe = BatchNormalization()(fe)
fe = Conv2D(64, (3,3), strides=(2, 2), padding='same', kernel_initializer=initialWeights)(fe)
fe = BatchNormalization()(fe)
fe = Conv2D(64, (3,3), strides=(2, 2), padding='same', kernel_initializer=initialWeights)(fe)
fe = BatchNormalization()(fe)
fe = Conv2D(64, (3,3), strides=(2, 2), padding='same', kernel_initializer=initialWeights)(fe)
fe = BatchNormalization()(fe)
fe = Conv2D(64, (3,3), strides=(2, 2), padding='same', kernel_initializer=initialWeights)(fe)
fe = BatchNormalization()(fe)
fe = Flatten()(fe)
fe = Dense(num_of_classes)(fe)
c_out_layer = Activation('softmax')(fe)
c_model = Model(in_image, c_out_layer)
c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
d_out_layer = Dense(1, activation='sigmoid')(fe)
d_model = Model(in_image,d_out_layer)
opt = Adam(lr=0.0002, beta_1=0.5)
d_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])


In [60]:
batch_size  = int(32)
X_val = np.array(X_val).astype('float32')
X_train = np.array(X_train).astype('float32')
y_train = np.array(y_train)
y_val = np.array(y_val)
CNN.fit(X_train, y_train, batch_size=32,epochs=75,validation_data=(X_val, y_val))
CNN.save('classifier_model.h5')

Epoch 1/75
39/39 [==============================] - 35s 886ms/step - loss: 10.5876 - accuracy: 0.0372 - val_loss: 10.6038 - val_accuracy: 0.0387
Epoch 2/75
39/39 [==============================] - 35s 886ms/step - loss: 10.5875 - accuracy: 0.0364 - val_loss: 10.6038 - val_accuracy: 0.0387
Epoch 3/75
39/39 [==============================] - 35s 888ms/step - loss: 10.5870 - accuracy: 0.0340 - val_loss: 10.6038 - val_accuracy: 0.0387
Epoch 4/75
39/39 [==============================] - 35s 885ms/step - loss: 10.5870 - accuracy: 0.0348 - val_loss: 10.6038 - val_accuracy: 0.0387
Epoch 5/75
39/39 [==============================] - 35s 886ms/step - loss: 10.5871 - accuracy: 0.0372 - val_loss: 10.6038 - val_accuracy: 0.0387
Epoch 6/75
39/39 [==============================] - 35s 888ms/step - loss: 10.5870 - accuracy: 0.0364 - val_loss: 10.6038 - val_accuracy: 0.0387
Epoch 7/75
39/39 [==============================] - 35s 886ms/step - loss: 10.5870 - accuracy: 0.0388 - val_loss: 10.6038 - val_ac

39/39 [==============================] - 35s 887ms/step - loss: 10.5867 - accuracy: 0.0493 - val_loss: 10.6038 - val_accuracy: 0.1355
Epoch 58/75
39/39 [==============================] - 34s 883ms/step - loss: 10.5867 - accuracy: 0.0744 - val_loss: 10.6038 - val_accuracy: 0.4452
Epoch 59/75
39/39 [==============================] - 34s 883ms/step - loss: 10.5867 - accuracy: 0.1415 - val_loss: 10.6038 - val_accuracy: 0.0387
Epoch 60/75
39/39 [==============================] - 35s 886ms/step - loss: 10.5868 - accuracy: 0.1504 - val_loss: 10.6038 - val_accuracy: 0.1355
Epoch 61/75
39/39 [==============================] - 35s 891ms/step - loss: 10.5867 - accuracy: 0.1584 - val_loss: 10.6038 - val_accuracy: 0.0452
Epoch 62/75
39/39 [==============================] - 34s 885ms/step - loss: 10.5867 - accuracy: 0.0719 - val_loss: 10.6038 - val_accuracy: 0.0387
Epoch 63/75
39/39 [==============================] - 34s 884ms/step - loss: 10.5867 - accuracy: 0.0719 - val_loss: 10.6038 - val_accurac

In [30]:
scores = c_model.predict(test_generator, 257/batch_size,workers=12)

array([5.65292775e-20, 1.80348865e-17, 7.03265154e-11, 1.07284805e-07,
       1.20792421e-09, 4.48849738e-01, 5.50354600e-01, 7.95611937e-04],
      dtype=float32)

In [5]:
#generator
# latent_dim = (128)
# in_lat = Input(shape=(latent_dim,))
# n_nodes = 128 * 8 * 8
# gen = Dense(n_nodes)(in_lat)
# gen = LeakyReLU(alpha=0.2)(gen)
# gen = keras.layers.Reshape((8, 8, 128))(gen)
# # upsample to 16x16*3
# gen = keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
# gen = LeakyReLU(alpha=0.2)(gen)
# # upsample to 32x32
# gen = keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
# gen = LeakyReLU(alpha=0.2)(gen)
# #64x64
# gen = keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
# gen = LeakyReLU(alpha=0.2)(gen)
# #128x128
# gen = keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
# gen = LeakyReLU(alpha=0.2)(gen)
# out_layer = Conv2D(1, (8,8), activation='tanh', padding='same')(gen)
# # define model
# generator = keras.Model(in_lat, out_layer)

generator = Sequential()
latent_dim = 256
# foundation for 4x4 image
n_nodes = 256 * 4 * 4
generator.add(Dense(n_nodes, input_dim=latent_dim))
#model.add(LeakyReLU(alpha=0.2))
generator.add(Reshape((4, 4, 256)))
# upsample to 8x8
generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', activation='relu', kernel_initializer=initialWeights))
#model.add(LeakyReLU(alpha=0.2))
# upsample to 16x16
generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', activation='relu', kernel_initializer=initialWeights))
#model.add(LeakyReLU(alpha=0.2))
# upsample to 32x32
generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', activation='relu', kernel_initializer=initialWeights))
generator.add(LeakyReLU(alpha=0.2))
# upsample to 64x64
generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', activation='relu', kernel_initializer=initialWeights))
generator.add(LeakyReLU(alpha=0.2))
# upsample to 128x128
generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', activation='relu', kernel_initializer=initialWeights))
generator.add(LeakyReLU(alpha=0.2))
# output layer
generator.add(Conv2D(3, (3,3), activation='tanh', padding='same'))

def define_gan(generator,discriminator ):
    generative = keras.Sequential()
    generative.add(generator)
    generative.add(discriminator)
    opt = Adam(lr=0.0002, beta_1=0.5)
    generative.compile(loss='binary_crossentropy', optimizer=opt)
    return generative
                    
            

from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint

def generate_latent_points(latent_dim, n_samples):
    #generate points in the latent space
    z_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = z_input.reshape(n_samples, latent_dim)
    return z_input
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict(z_input)
    y = zeros((n_samples, 1))
    return images, y
 

In [36]:
log_dir = "/home/josheen/projects/def-vinay/josheen/DL/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
my_callbacks = [tensorboard_callback,keras.callbacks.ModelCheckpoint(filepath = '/home/josheen/projects/def-vinay/josheen/DL/model.{epoch:02d}.h5'), keras.callbacks.EarlyStopping(patience = 2)]

discriminator.fit(train_generator, epochs = 35, steps_per_epoch = 10, validation_data= val_generator, validation_steps = 200, callbacks = my_callbacks)
discriminator.save_weights('discrim1.h5')

Epoch 1/35
10/10 [==============================] - ETA: 0s - loss: 0.1370 - accuracy: 0.9750

W1119 20:14:47.459025 47953444966080 callbacks.py:1689] Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


10/10 [==============================] - 8s 850ms/step - loss: 0.1370 - accuracy: 0.9750
Epoch 2/35
10/10 [==============================] - ETA: 0s - loss: 0.1150 - accuracy: 0.9781

W1119 20:14:57.141201 47953444966080 callbacks.py:1689] Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


10/10 [==============================] - 8s 820ms/step - loss: 0.1150 - accuracy: 0.9781
Epoch 3/35
10/10 [==============================] - ETA: 0s - loss: 0.1784 - accuracy: 0.9625

...

10/10 [==============================] - 7s 656ms/step - loss: 0.1550 - accuracy: 0.9627
Epoch 34/35
10/10 [==============================] - ETA: 0s - loss: 0.1181 - accuracy: 0.9750

W1119 20:18:03.692818 47953444966080 callbacks.py:1689] Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


10/10 [==============================] - 4s 413ms/step - loss: 0.1181 - accuracy: 0.9750
Epoch 35/35
10/10 [==============================] - ETA: 0s - loss: 0.1288 - accuracy: 0.9750

W1119 20:18:08.372949 47953444966080 callbacks.py:1689] Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


10/10 [==============================] - 4s 407ms/step - loss: 0.1288 - accuracy: 0.9750


In [42]:
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return [X, labels], y

def generate_unsupervised_samples(dataset, n_samples=8000):
    ix = randint(0, dataset.shape[0], n_samples)
    # select images and labels
    y = ones((n_samples, 1))
    X = dataset[ix]
    return X, y
    
def select_supervised_samples(dataset, labels, n_samples=104, n_classes=8):
    n_per_class = int(n_samples / n_classes)
    y = labels
    X_list, y_list = list(), list()
    for i in range(n_classes):
        # get all images for this class
        X_with_class = labels.index[labels['y'] == i].tolist()
        # choose random instances
        ix = randint(0, len(X_with_class), n_per_class)
        # add to list
        [X_list.append(dataset[j]) for j in ix]
        [y_list.append(i) for j in ix]
    return np.asarray(X_list), np.asarray(y_list)
 

def summarize_performance(epoch, g_model, c_model, latent_dim, dataset,labels, n_samples=100):
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    # evaluate the classifier model
    X, y = dataset, labels
    _, acc = c_model.evaluate(X, y, verbose=0)
    print('Classifier Accuracy: %.3f%%' % (acc * 100))

def train(g_model, d_model,c_model, gan_model, dataset, labels,unlabeled, latent_dim, n_epochs=30, n_batch=10):
    X_sup, y_sup = select_supervised_samples(dataset,labels)
    print(X_sup.shape, y_sup.shape)
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps))
    # manually enumerate epochs
    for i in range(n_steps):
        # update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], half_batch)
        c_loss, c_acc = c_model.train_on_batch(Xsup_real, ysup_real)
        # update unsupervised discriminator (d)
        X_real, y_real = generate_unsupervised_samples(unlabeled, half_batch)
        d_loss1 = d_model.train_on_batch(X_real, y_real)
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        d_loss2 = d_model.train_on_batch(X_fake, y_fake)
        # update generator (g)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
        # summarize loss on this batch
        # evaluate the model performance every so often
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, c_model, latent_dim, dataset, labels)
    c_model.save('classifier_model.h5')
    g_model.save('GAN_model.h5')
    d_model.save('discrim2.h5')

In [28]:
df = pd.read_csv('/home/josheen/projects/def-vinay/josheen/DL/saved_df.csv')
left=115
top=35
right=243
bottom=163

train_files = []
y_train = []
dataset = []
for index, row in df.iterrows():
    y_train.append(row['Recovery']) 
    img = load_img(row['Address'])  # this is a PIL image
    img1 = img.crop((left,top,right,bottom))
    x = img_to_array(img1)
    x = (x - 255.0) / 255.0
    dataset.append(x)


In [43]:
dataset = np.array(dataset)
unsupervised_data = np.array(unsupervised_data)
labels = pd.DataFrame(y_train,columns = ['y'])
generator.load_weights('GAN_Model.h5')
d_model.load_weights('discrim2.h5')
c_model.load_weights('classifier_model.h5')
generative = define_gan(generator,d_model)
train(generator,d_model, c_model, generative, dataset,labels, unsupervised_data, 256)

(104, 128, 128, 3) (104,)
n_epochs=30, n_batch=10, 1/2=5, b/e=12, steps=360
Classifier Accuracy: 17.244%
Classifier Accuracy: 11.950%
Classifier Accuracy: 25.886%
Classifier Accuracy: 6.384%
Classifier Accuracy: 9.576%
Classifier Accuracy: 3.348%
Classifier Accuracy: 20.670%
Classifier Accuracy: 18.256%
Classifier Accuracy: 8.252%
Classifier Accuracy: 10.860%
Classifier Accuracy: 9.420%
Classifier Accuracy: 4.399%
Classifier Accuracy: 10.549%
Classifier Accuracy: 5.527%
Classifier Accuracy: 7.162%
Classifier Accuracy: 5.917%
Classifier Accuracy: 7.007%
Classifier Accuracy: 4.087%
Classifier Accuracy: 25.730%
Classifier Accuracy: 5.566%
Classifier Accuracy: 4.632%
Classifier Accuracy: 4.282%
Classifier Accuracy: 21.643%
Classifier Accuracy: 26.508%
Classifier Accuracy: 4.243%
Classifier Accuracy: 5.060%
Classifier Accuracy: 24.445%
Classifier Accuracy: 19.112%
Classifier Accuracy: 27.715%
Classifier Accuracy: 26.547%


In [40]:
save_dir = '/home/josheen/projects/def-vinay/josheen/DL/generated_images'
img_loc = pd.read_csv('/home/josheen/projects/def-vinay/josheen/DL/saved_df.csv')

generate_data = ImageDataGenerator(
    rescale = 1./255, rotation_range = 5, width_shift_range = 0.2, height_shift_range = 0.2, shear_range=0.2,
    zoom_range=0.2, horizontal_flip = True, fill_mode='nearest')

for index, row in img_loc.iterrows():
    image = load_img(row['Address'])
    image = np.expand_dims(img_to_array(image),0)
    generate_data.fit(image)
    for x, val in zip(generate_data.flow(image,save_to_dir=save_dir,save_prefix='aug',save_format='png'),range(5)):
        pass

In [41]:
left=115
top=35
right=243
bottom=163
unsupervised_data = []
for file in os.listdir('/home/josheen/projects/def-vinay/josheen/DL/generated_images'):
    img1 = img.crop((left,top,right,bottom))
    x = img_to_array(img1)
    x = (x - 255.0) / 255.0
    unsupervised_data.append(x)

In [30]:
num_classes = 8
categorical_train = []
for val in y_train:
    if val <= 5:
        categorical_train.append(0)
    elif val > 5 and val <= 10:
        categorical_train.append(1)
    elif val > 10 and val<= 15:
        categorical_train.append(2)
    elif val > 15 and val<= 20:
        categorical_train.append(3)
    elif val > 20 and val<= 25:
        categorical_train.append(4)
    elif val > 25 and val <= 30:
        categorical_train.append(5)
    elif val > 30 and val <= 35:
        categorical_train.append(6)
    elif val >35:
        categorical_train.append(7)
y_train = categorical_train

In [39]:
print(len(unsupervised_data))

2682
